This notebook is to batch process adaptations and analyse results

In [2]:
# Imports
import sys
sys.path = []
sys.path.append(r'C:\repos\ci_adapt')
sys.path.append(r'C:\repos\snkit\src')
sys.path.append(r'C:\repos\ra2ce')
sys.path.append(r'C:\repos\ra2ce_multi_network')
sys.path.extend([
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\python312.zip',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\DLLs',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\win32',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\win32\lib',
    r'C:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\Pythonwin'
])




In [3]:

sys.path.append(r'C:\repos\snkit\src')
sys.path.append(r'C:\repos\ra2ce')
sys.path.append(r'C:\repos\ra2ce_multi_network')
from ci_adapt_utilities import *

In [4]:
# Load configuration with ini file (created running config.py)
# config_file=r'C:\repos\ci_adapt\config_ci_adapt_test.ini'
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
hazard_type, infra_type, country_code, country_name, hazard_data_subfolders, asset_data, vulnerability_data = load_config(config_file)

In [5]:
# Define paths and load data
# data_path = Path(pathlib.Path(r'C:\Users\peregrin\OneDrive - Stichting Deltares\Documents\PhD Daniel Shared\Papers\Paper 1 - Adaptation Framework\Data\test'))
data_path = Path(r'C:\Data')
interim_data_path = data_path / 'interim' / 'collected_flood_runs'
assets, geom_dict, miraca_colors, return_period_dict, adaptation_unit_costs, rp_spec_priority, average_road_cost_per_ton_km, average_train_cost_per_ton_km, average_train_load_tons = startup_ci_adapt(data_path, config_file)


47210 assets loaded.


In [6]:
# Load data from baseline impact assessment
shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'shortest_paths.pkl', 'rb'))
disrupted_edges_by_basin = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_edges_by_basin.pkl', 'rb'))
graph_r0 = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'graph_0.pkl', 'rb'))
disrupted_shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_shortest_paths.pkl', 'rb'))
event_impacts = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'event_impacts.pkl', 'rb'))
full_flood_event=pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'full_flood_event.pkl', 'rb'))
all_disrupted_edges = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'all_disrupted_edges.pkl', 'rb'))
collect_output = pickle.load(open(data_path / 'interim' / 'collected_flood_runs' / 'sample_collected_run.pkl', 'rb'))
print('Loaded data from baseline impact assessment')
graph_v0=create_virtual_graph(graph_r0)
graph_v=graph_v0.copy()

Loaded data from baseline impact assessment
Creating virtual graph...
Success: only int type values


In [7]:
adaptations={}
adaptations['baseline'] = {'l1_l2_adapt_path': None, 'added_links':[], 'l4_adapt_path': None}
adaptations['l1_trib'] = {'l1_l2_adapt_path': data_path/r'input\adaptations\l1_tributary.geojson', 'added_links':[], 'l4_adapt_path': None}
adaptations['l2_trib'] = {'l1_l2_adapt_path': data_path/r'input\adaptations\l2_tributary.geojson', 'added_links':[], 'l4_adapt_path': None}
adaptations['l3_trib'] = {'l1_l2_adapt_path': None, 'added_links':[(4424116, 219651487), (219651487, 111997047)], 'l4_adapt_path': None}
adaptations['l3_trib_a'] = {'l1_l2_adapt_path': None, 'added_links':[(4424116, 219651487)], 'l4_adapt_path': None}
adaptations['l3_trib_b'] = {'l1_l2_adapt_path': None, 'added_links':[(219651487, 111997047)], 'l4_adapt_path': None}
adaptations['l4_trib'] = {'l1_l2_adapt_path': None, 'added_links':[], 'l4_adapt_path': data_path/r'input\adaptations\l4_tributary.geojson'}
adaptations['l1_rhine'] = {'l1_l2_adapt_path': data_path/r'input\adaptations\l1_rhine.geojson', 'added_links':[], 'l4_adapt_path': None}
adaptations['l2_rhine'] = {'l1_l2_adapt_path': data_path/r'input\adaptations\l2_rhine.geojson', 'added_links':[], 'l4_adapt_path': None}
adaptations['l3_rhine'] = {'l1_l2_adapt_path': None, 'added_links':[(112044105, 110947346)], 'l4_adapt_path': None}
adaptations['l4_rhine'] = {'l1_l2_adapt_path': None, 'added_links':[], 'l4_adapt_path': data_path/r'input\adaptations\l4_rhine.geojson'}

In [8]:
# Store results in dictionaries
direct_damages_adapted_dict = {}
indirect_damages_adapted_dict = {}
indirect_damages_adapted_full_dict = {}
adaptation_costs={}
adapted_assets_dict = {}

# Print adaptations that will be run
print(f"Processing {len(adaptations)} scenarios:")
for adapt_id in adaptations.keys():
    print('- ',adapt_id)

# for adapt_id in tqdm(adaptations.keys(), desc='Adaptation runs', total=len(adaptations)):
for adapt_id in adaptations.keys():
    adaptations_df_path = data_path / 'interim' / 'adaptations' / f'{adapt_id}_adaptations.csv'

    if adaptations_df_path.exists():
        print(f"Adaptation {adapt_id} already processed. Skipping.")
        continue

    # reset variables
    graph_v=graph_v0.copy()

    # Load adaptations dictionary to the relevant variables
    l1_l2_adapt_path = adaptations[adapt_id]['l1_l2_adapt_path']
    added_links = adaptations[adapt_id]['added_links']  
    l4_adapt_path = adaptations[adapt_id]['l4_adapt_path']

    # Load adaptation data
    if l1_l2_adapt_path is not None:
        adapted_area = gpd.read_file(l1_l2_adapt_path).to_crs(3857)
    else:
        adapted_area = None
    if l4_adapt_path is not None:
        adapted_route_area = gpd.read_file(l4_adapt_path).to_crs(3857)
    else:
        adapted_route_area = None

    # Apply adaptations
    adapted_assets, adaptations_df, demand_reduction_dict, l3_adaptation_costs = apply_adaptations(adapted_area, assets, collect_output, interim_data_path, rp_spec_priority, adaptation_unit_costs, shortest_paths, graph_v, added_links, adapted_route_area)

    # Calculate l1 adaptation costs
    local_haz_path=Path(r'N:\Projects\11209000\11209175\B. Measurements and calculations\Data\basin_flood_hazard_maps')#Test data: data_path/r'Floods\Germany\basin_intersections'
    l1_adaptation_costs=calculate_l1_costs(local_haz_path, interim_data_path, adapted_area, adaptation_unit_costs, adapted_assets) 
    print(f"l1_adaptation_costs: {l1_adaptation_costs}")

    # Run adapted damages for individual hazard maps
    direct_damages_adapted, indirect_damages_adapted, adaptation_run_full, l2_adaptation_costs, overlay_assets_lists = run_adapted_damages(data_path, config_file, collect_output, disrupted_edges_by_basin, interim_data_path, assets, geom_dict, adapted_assets, adaptations_df, rp_spec_priority, adaptation_unit_costs, shortest_paths, graph_v, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km, demand_reduction_dict)

    # Run adapted damages for full flood event
    indirect_damages_adapted_full = calculate_indirect_dmgs_fullflood(full_flood_event, overlay_assets_lists, adaptation_run_full, assets, all_disrupted_edges, shortest_paths, graph_v, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km, demand_reduction_dict)


    # Fill in missing values in dictionaries
    for hazard_map in collect_output.keys():
        if direct_damages_adapted[hazard_map]=={}:
            direct_damages_adapted[hazard_map]=collect_output[hazard_map]
        if indirect_damages_adapted[hazard_map]=={}:
            indirect_damages_adapted[hazard_map]=event_impacts[hazard_map] if hazard_map in event_impacts.keys() else 0.0
    
    # Store results in dictionaries
    direct_damages_adapted_dict[adapt_id] = direct_damages_adapted
    indirect_damages_adapted_dict[adapt_id] = indirect_damages_adapted
    indirect_damages_adapted_full_dict[adapt_id] = indirect_damages_adapted_full
    adapted_assets_dict[adapt_id] = adapted_assets
    adaptation_costs[adapt_id] = {'l1': l1_adaptation_costs, 
                                  'l2': l2_adaptation_costs, 
                                  'l3': l3_adaptation_costs}
    adaptations_df.to_csv(data_path / 'interim' / 'adaptations' / f'{adapt_id}_adaptations.csv')
  

Processing 11 scenarios:
-  baseline
-  l1_trib
-  l2_trib
-  l3_trib
-  l3_trib_a
-  l3_trib_b
-  l4_trib
-  l1_rhine
-  l2_rhine
-  l3_rhine
-  l4_rhine
Adaptation baseline already processed. Skipping.
Applying adaptation:  fwall_nahe
Level 1 adaptation
Troubleshooting 1
Troubleshooting 1.1
   id   prot_area rp_spec  adapt_level adapt_size adapt_unit  \
0   1  fwall_nahe       L            1       6500          m   

                                            geometry  \
0  LINESTRING (878179.421 6440400.990, 878286.712...   

                                            buffered  
0  POLYGON ((878178.4270064099 6440401.040473994,...  
Troubleshooting 1.2
0 id                                                             1
prot_area                                             fwall_nahe
rp_spec                                                        L
adapt_level                                                    1
adapt_size                                                  6500
adapt_u

In [9]:
# Report output dataframe
output_df = pd.DataFrame.from_dict([direct_damages_adapted_dict, indirect_damages_adapted_dict, indirect_damages_adapted_full_dict, adapted_assets_dict, adaptation_costs])
output_df


,l1_trib,l1_rhine
0,"{'flood_DERP_RW_H_4326_2080410170': {2736: (0,...","{'flood_DERP_RW_H_4326_2080410170': {2736: (0,..."
1,{'flood_DERP_RW_H_4326_2080410170': 1343904.97...,{'flood_DERP_RW_H_4326_2080410170': 1343904.97...
2,"{'flood_DERP_RW_H': 102200714.60236514, 'flood...","{'flood_DERP_RW_H': 92963532.05601165, 'flood_..."
3,osm_id asset name gauge el...,osm_id asset name gau...
4,"{'l1': {0: 266240624.64660648}, 'l2': {}, 'l3'...","{'l1': {0: 185930111.48819607}, 'l2': {}, 'l3'..."


In [10]:
# Save results
for adapt_id in adaptations.keys():
    if not adapt_id in direct_damages_adapted_dict.keys():
        continue
    direct_damages_adapted_path = data_path / 'output' / f'adapted_direct_damages_{adapt_id}.pkl'
    indirect_damages_adapted_path = data_path / 'output' / f'adapted_indirect_damages_{adapt_id}.pkl'
    indirect_damages_adapted_full_path = data_path / 'output' / f'adapted_indirect_damages_full_{adapt_id}.pkl'
    # adaptations_df_path = data_path / 'output' / f'adaptations_{adapt_id}.csv'
    adapted_assets_path = data_path / 'output' / f'adapted_assets_{adapt_id}.pkl'
    adaptation_costs_path = data_path / 'output' / f'adaptation_costs_{adapt_id}.pkl'
    adaptations_csv_path = data_path / 'interim' / 'adaptations' / f'{adapt_id}_adaptations.csv'


    adaptations_df = pd.DataFrame.from_dict(adaptations[adapt_id])
    

    with open(direct_damages_adapted_path, 'wb') as f:
        pickle.dump(direct_damages_adapted_dict[adapt_id], f)
    with open(indirect_damages_adapted_path, 'wb') as f:
        pickle.dump(indirect_damages_adapted_dict[adapt_id], f)
    with open(indirect_damages_adapted_full_path, 'wb') as f:
        pickle.dump(indirect_damages_adapted_full_dict[adapt_id], f)    
    with open(adapted_assets_path, 'wb') as f:
        pickle.dump(adapted_assets_dict[adapt_id], f)
    with open(adaptation_costs_path, 'wb') as f:
        pickle.dump(adaptation_costs[adapt_id], f)
    print(f'Saved results for adaptation: {adapt_id}')

Saved results for adaptation: l1_trib
Saved results for adaptation: l1_rhine


In [10]:
# load results for adaptation
for adapt_id in adaptations.keys():
    direct_damages_adapted_path = data_path / 'output' / f'adapted_direct_damages_{adapt_id}.pkl'
    indirect_damages_adapted_path = data_path / 'output' / f'adapted_indirect_damages_{adapt_id}.pkl'
    indirect_damages_adapted_full_path = data_path / 'output' / f'adapted_indirect_damages_full_{adapt_id}.pkl'
    adaptation_costs_path = data_path / 'output' / f'adaptation_costs_{adapt_id}.pkl'
    adaptations_df_path = data_path / 'interim' / 'adaptations' / f'{adapt_id}_adaptations.csv'

    # Load data from files
    with open(direct_damages_adapted_path, 'rb') as f:
        direct_damages_adapted_dict[adapt_id] = pickle.load(f)
    with open(indirect_damages_adapted_path, 'rb') as f:
        indirect_damages_adapted_dict[adapt_id] = pickle.load(f)
    with open(indirect_damages_adapted_full_path, 'rb') as f:
        indirect_damages_adapted_full_dict[adapt_id] = pickle.load(f)
    with open(adaptation_costs_path, 'rb') as f:
        adaptation_costs[adapt_id] = pickle.load(f)

# load bcr from csv
bcr_path = data_path / 'output' / f'bcr_df.csv'
try:
    bcr_df = pd.read_csv(bcr_path)
except FileNotFoundError:
    print(f"BCR file not found at {bcr_path}")
    bcr_df = pd.DataFrame({'adaptation': list(adaptations.keys())})

bcr_df    

bcr_df.sort_values('bcr_upper', ascending=False)
# bcr_df.head(5)

BCR file not found at C:\Data\output\bcr_df.csv


KeyError: 'bcr_upper'

In [14]:
# Clean-up cell
# delete interim files to allow output recalculation
for file in os.listdir(data_path / 'interim' / 'adaptations'):
    if file.endswith('.csv'):
        # os.remove(data_path / 'interim' / 'adaptations' / file)
        print(file)
    # os.remove(data_path / 'interim' / 'adaptations' / file)

baseline_adaptations.csv
l1_rhine_adaptations.csv
l1_trib_adaptations.csv
l2_rhine_adaptations.csv
l2_trib_adaptations.csv
l3_rhine_adaptations.csv
l3_trib_adaptations.csv
l4_rhine_adaptations.csv
l4_trib_adaptations.csv
